# pca Face Recognition system

In [77]:
import os
import zipfile
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

In [78]:
# Step 1: Extract zip file
zip_file = "/content/file.zip"   # <-- your dataset zip file
extract_path = "faces_data_file"
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [79]:
# Step 2: Load dataset
def load_images_from_folder(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (100, 100))  # resize for consistency
            images.append(img.flatten())
            labels.append(label)
    return images, labels

In [80]:
X, y = [], []
classes = os.listdir(extract_path)  # ['one','two','three','four']

In [81]:
classes

['two', 'four', 'one', 'three']

In [82]:
for idx, cls in enumerate(classes):
    folder = os.path.join(extract_path, cls)
    imgs, labels = load_images_from_folder(folder, cls)
    X.extend(imgs)
    y.extend(labels)

In [83]:
X = np.array(X)
y = np.array(y)

print("Dataset shape:", X.shape)

Dataset shape: (32, 10000)


In [91]:
len(y)

32

In [92]:
# Step 3: Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled


array([[-0.17646863,  0.22260856,  0.2413197 , ..., -0.44002547,
        -0.49270123, -0.63907089],
       [ 0.37356347,  0.25735721,  0.2413197 , ..., -2.6939689 ,
        -2.99448336, -2.77603698],
       [-0.21313744,  0.22260856,  0.2413197 , ..., -0.56665151,
        -0.62300239, -0.73846466],
       ...,
       [ 0.37356347,  0.25735721,  0.2413197 , ...,  0.82623488,
         0.62788868,  0.72759347],
       [ 0.37356347,  0.25735721,  0.2413197 , ...,  0.82623488,
         0.62788868,  0.72759347],
       [ 0.37356347,  0.25735721,  0.2413197 , ...,  0.82623488,
         0.62788868,  0.72759347]])

In [95]:
# Step 4: PCA
pca = PCA(n_components=30)  # keep 50 eigenfaces
X_pca = pca.fit_transform(X_scaled)
#X_pca
print("PCA transformed shape:", X_pca.shape)

PCA transformed shape: (32, 30)


In [87]:
# Step 5: Store class means in PCA space
class_means = {}
for cls in np.unique(y):
    class_means[cls] = np.mean(X_pca[y == cls], axis=0)

class_means

{np.str_('four'): array([-1.59467986e+01, -1.33344672e+01,  6.12493687e+00, -1.88009001e+01,
        -2.61236426e+01,  1.16780872e+01,  4.28741367e+00, -1.41002191e+00,
        -1.23078077e+00, -3.47747828e+00,  5.17184744e-01,  9.03334804e-01,
         5.91865051e-01,  2.01049391e-01,  5.85534686e-01, -3.30676666e-01,
        -8.28422034e-01, -1.44372825e-01,  5.43069654e-02,  1.32300464e+00,
        -2.56763981e-01, -1.97397781e-02, -2.05812015e-01, -4.45501755e-02,
        -1.50375722e-02,  3.53369480e-02, -3.25008664e-02, -6.90618141e-02,
         1.62394682e-02,  2.65969876e-14]),
 np.str_('one'): array([-1.19017288e+01,  2.34685670e+01,  2.19909112e+01,  3.76893618e+01,
        -4.26198019e+00, -3.20521129e+00, -4.96467806e-01,  2.03063995e+00,
         2.76568907e+00,  4.20375780e+00, -9.42113672e-01, -2.02931411e+00,
         1.54038565e+00,  1.78409365e-01, -5.59321447e-01, -1.34919021e-01,
        -2.05548999e-01, -3.36787723e-01, -3.91383923e-01,  1.33046585e-01,
        -1.

In [88]:

model_data = {
    "scaler": scaler,
    "pca": pca,
    "class_means": class_means
}

with open("pca_face_model.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("Model saved as pca_face_model.pkl")

Model saved as pca_face_model.pkl


In [89]:

def predict_face(img_path, model_path="pca_face_model.pkl"):
    with open(model_path, "rb") as f:
        model = pickle.load(f)

    scaler = model['scaler']
    pca = model['pca']
    class_means = model['class_means']

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (100, 100))
    img_flat = img.flatten().reshape(1, -1)

    img_scaled = scaler.transform(img_flat)
    img_pca = pca.transform(img_scaled)

    # Compare with class means
    min_dist = float("inf")
    predicted_class = None

    for cls, mean_vec in class_means.items():
        dist = euclidean_distances(img_pca, mean_vec.reshape(1, -1))[0][0]
        if dist < min_dist:
            min_dist = dist
            predicted_class = cls

    return predicted_class


test_image = "/content/subject03.sad.png"
predicted = predict_face(test_image)
print("Predicted Class:", predicted)

Predicted Class: three


In [90]:
import joblib

# Save
joblib.dump(model_data, "pca_face_model.joblib")

# Load
model_data = joblib.load("pca_face_model.joblib")
